# HRRR Forecast - one hour
This notebook demonstrates how to access one GRIB2 file in xarray. 

In [ ]:
import warnings
import fsspec

import datetime as dt
import xarray as xr
import hvplot.xarray  # noqa

warnings.filterwarnings("ignore")

In [ ]:
fs = fsspec.filesystem("s3", anon=True, skip_instance_cache=True)

today_files = fs.glob(
    f"s3://noaa-hrrr-bdp-pds/hrrr.{dt.datetime.utcnow():%Y%m%d}/conus/*wrfsfcf01.grib2"
)
last_file = today_files[-1]
last_file

In [ ]:
file = fsspec.open_local(
    f"simplecache::s3://{last_file}",
    s3={"anon": True},
    filecache={"cache_storage": "/tmp/files"},
)
file

In [ ]:
ds = xr.open_dataset(
    file,
    engine="cfgrib",
    filter_by_keys={"typeOfLevel": "heightAboveGround", "level": 2},
)
ds.t2m

Hvplot wants lon [-180,180], not [0,360]:

In [ ]:
ds = ds.assign_coords(longitude=(((ds.longitude + 180) % 360) - 180))

In [ ]:
ds.t2m.hvplot.quadmesh(x="longitude", y="latitude", cmap="turbo", rasterize=True, geo=True, tiles="OSM")